In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# FILES
mlp_stats = np.load("./results/v1/stats.npy")
cnn2_stats = np.load("./results/v2_2/ranks.npy")
cnne_stats = np.load("./results/v4/rank_stats.npy")

# print(mlp_stats.shape, cnne_stats.shape, cnne_stats_inputs.shape)

In [7]:
# Labels and print functions
mlp_label = ["P2_X", "P2_Y", "P1_O1","P1_O2","P1_Onion","P1_Dish", "P1_Soup", "P2_O1","P2_O2","P2_Onion","P2_Dish", "P2_Soup","TP_X","TP_Y","TO_X","TO_Y","TD_X","TD_Y","TS_X","TS_Y"]
cnne_ch_label = ["Map","Bench","Pot","Onion Box","Dish Box","Serve","Pot Onions","Pot Tick","Onion(s)","Dish(es)", "Soup(s)","P1 O1","P1 O2","P1 O3","P1 O4"]#,"P2 O1","P2 O2","P2 O3","P2 O4"]
tile_label = ["Bench","Pot","Onion","Dish","Serve","P1","P2","Empty"]
def print_lists(orders,labels,col_names):
    #HEADER
    header=f"    "
    for name in col_names:
        header += f"{name:<15}"
    print(header)

    L = len(orders[0])
    for i in range(L):
        row=f"{i+1:<4}"
        for order in orders:
            row+= f"{labels[order[i]]:<15}"
        
        print(row)

In [8]:
#Schluze ranking method
def schluze(stats):
    W  = len(stats[0])
    d_grid = np.zeros([W,W])
    for row in stats:
        for i in range(W):
            if i<W:
                for j in range(i+1,W):
                    item1 = row[i]
                    item2 = row[j]
                    # print(item1,item2)
                    if item1>item2:
                        d_grid[i,j]+=1
                    elif item2>item1:
                        d_grid[j,i]+=1
                    else:
                        print("idk")
                        d_grid[i,j]+=1
    # print(d_grid)

    p_grid = np.zeros([W,W])
    for i in range(W):
        for j in range(W):
            if i!=j:
                if d_grid[i,j] > d_grid[j,i]:
                    p_grid[i,j] = d_grid[i,j]
                else:
                    p_grid[i,j]=0

    for i in range(W):
        for j in range(W):
            if i!=j:
                for k in range(W):
                    if i!=k and j!=k:
                        p_grid[j,k] = max(p_grid[j,k],min(p_grid[j,i],p_grid[i,k]))
    

    # print(p_grid)

    w_grid = np.zeros([W,W])
    for i in range(W):
        if i<W:
            for j in range(i+1,W):
                item1 = p_grid[i,j]
                item2 = p_grid[j,i]

                # print(item1,item2)
                if item1>item2:
                    w_grid[i,j]=1
                elif item2>item1:
                    w_grid[j,i]=1
                else:
                    print("idk2")
    # print(w_grid)

    w_sum = np.sum(w_grid,axis=0)
    return w_sum


In [6]:
# MLP Rank addition combination
mlp_rank_sum = np.zeros(mlp_stats.shape[1])
for row in mlp_stats:
    # row = np.round(row,2)
    row_ranks = row.argsort().argsort()
    mlp_rank_sum += row_ranks
mlp_rank_order = (-mlp_rank_sum).argsort()

# MLP Rank mean combination
mlp_grad_sum = np.linalg.norm(mlp_stats,axis=1)
mlp_grad_norm = mlp_stats / mlp_grad_sum[:, np.newaxis]

mlp_grad_mean = np.mean(mlp_grad_norm,axis=0)
mlp_mean_order = (-mlp_grad_mean).argsort()

# MLP Rank schluze combination
mlp_schluze_rank = schluze(mlp_stats)
mlp_schluze_order = mlp_schluze_rank.argsort()

# Print orders
col_names = ["AddSort","Mean","Schluze"]

print_lists([mlp_rank_order, mlp_mean_order, mlp_schluze_order],mlp_label,col_names)

    AddSort        Mean           Schluze        
1   P1_Soup        P1_Soup        P1_Soup        
2   P1_O1          P1_O1          P1_O1          
3   P1_O2          P1_O2          P1_O2          
4   TP_Y           TP_X           TP_Y           
5   TP_X           TP_Y           TP_X           
6   TS_Y           TS_Y           TS_Y           
7   TO_X           TO_Y           TO_X           
8   TD_Y           TO_X           TO_Y           
9   TO_Y           TD_Y           TD_Y           
10  P2_O2          P2_O2          P2_O2          
11  TD_X           TS_X           TD_X           
12  TS_X           TD_X           P2_O1          
13  P2_O1          P2_O1          TS_X           
14  P2_X           P2_X           P2_X           
15  P2_Y           P2_Y           P2_Y           
16  P2_Onion       P2_Onion       P2_Onion       
17  P1_Dish        P1_Dish        P1_Dish        
18  P2_Dish        P2_Dish        P2_Dish        
19  P1_Onion       P1_Onion       P1_Onion       


In [20]:
# CNNe Rank schluze combination
# Channel Analysis

cnne_flat_stats = cnne_stats.reshape(cnne_stats.shape[0],cnne_stats.shape[1],cnne_stats.shape[2]*cnne_stats.shape[3])

cnne_stats_ch_max = np.max(cnne_flat_stats,axis=2)
cnne_stats_ch_std = np.std(cnne_flat_stats,axis=2)
# print(cnne_flat_stats.shape, cnne_stats_ch_max.shape, cnne_stats_ch_std.shape)

cnne_schluze_max_rank = schluze(cnne_stats_ch_max)
cnne_schluze_max_order = cnne_schluze_max_rank.argsort()

cnne_schluze_std_rank = schluze(cnne_stats_ch_std)
cnne_schluze_std_order = cnne_schluze_std_rank.argsort()

# Print orders
col_names = ["Schluze Max","Schluze Std"]

print_lists([cnne_schluze_max_order, cnne_schluze_std_order],cnne_ch_label,col_names)

    Schluze Max    Schluze Std    
1   P1 O1          Pot Tick       
2   P1 O3          P1 O1          
3   P1 O4          P1 O3          
4   Pot Tick       P1 O4          
5   P1 O2          P1 O2          
6   Dish(es)       Dish(es)       
7   Onion(s)       Onion(s)       
8   Soup(s)        Soup(s)        
9   Pot Onions     Pot Onions     
10  Serve          Serve          
11  Dish Box       Dish Box       
12  Onion Box      Onion Box      
13  Pot            Pot            
14  Bench          Bench          
15  Map            Map            


In [10]:
#Stats generated in respective file to reduce memory requirements
# Each row has a sequency of characters representing tiles in order of saliency (highest to lowest)
# Used max across colour channels and downsampled to grid size.
def rank_stats(ss_arr, title):
    #Reformat data for schluze application
    # note this results in encoding before the schluze function and decoding within the function
    # so this can be optimised to just 1 direct encoding if needed

    #No duplicate items in sequence
    # 0 - Bench (X) 0/1
    # 1 - Pot 0/1
    # 2 - Onion Source 0/1
    # 3 - Dish Source 0/1
    # 4 - Serving 0/1
    # 5 - Player 1
    # 6 - Player 2
    # 7 - None/Empty space

    chars = 'XPODS12E'
    idx_dict = {chars[i]: i for i in range(len(chars))}
    ss_arr_order = np.zeros(ss_arr.shape)
    for i in range(len(ss_arr)):
        for j in range(len(ss_arr[i])):
            ss_arr_order[i][idx_dict[ss_arr[i][j]]] = len(ss_arr[i]) - j - 1

    #Schluze ranking
    cnn2_schluze_max_rank = schluze(ss_arr_order)
    cnn2_schluze_max_order = cnn2_schluze_max_rank.argsort()

    # Print orders
    col_names = [title]
    print_lists([cnn2_schluze_max_order],tile_label,col_names)


    #Displays distribution of ranking
    cnn_ss_no_dupes = np.zeros([8,8],dtype="int64")
    for ss2 in ss_arr_order:
        for i in range(len(ss2)):
            ranking = int(ss2[i])
            cnn_ss_no_dupes[7-ranking,i]+=1
    print(cnn_ss_no_dupes)

In [11]:
rank_stats(cnn2_stats, "CNN2")
#The array represents the ranking of items (first row is number of occurences of a tile in rank 1, etc. The column indicates trhe type of tile. This is used in the poster.)

    CNN2           
1   P1             
2   P2             
3   Pot            
4   Bench          
5   Dish           
6   Empty          
7   Onion          
8   Serve          
[[    5    16    14   192     0 12948   483     0]
 [ 1696  3426   803  2768    28   661  3704   572]
 [ 2278  2764  1591  2450   133    47  2621  1774]
 [ 2898  1699  1453  1821   119     2  2525  3141]
 [ 3586  1741  1660  1260   202     0  1982  3227]
 [ 2095  1952  3298  1442   563     0  1376  2932]
 [ 1039  1800  3307  2075  2775     0   827  1835]
 [   61   260  1532  1650  9838     0   140   177]]


In [18]:
rank_stats(cnne_stats, "Encoded CNN")
#Ranked by backpropagated gradients, may not be valid for encoded CNN due to different input types
#The array represents the ranking of items (first row is number of occurences of a tile in rank 1, etc. The column indicates trhe type of tile)

    Encoded CNN    
1   Empty          
2   P1             
3   Bench          
4   P2             
5   Serve          
6   Onion          
7   Dish           
8   Pot            
[[1869   34  133  137  329 4748 1366 5042]
 [3283  108  362  258  762 2913 1169 4803]
 [4829  344  828  588 1327 1953 1403 2386]
 [2506 1103 1908 1376 2368 1500 1942  955]
 [ 819 2156 2438 2236 2271 1183 2219  336]
 [ 294 2871 2984 2567 2091  709 2044   98]
 [  51 3323 2882 3042 2301  445 1578   36]
 [   7 3719 2123 3454 2209  207 1937    2]]
